<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Scrape Album from Artists </span>

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
%run py_utils/scraping_utils.ipynb
%run py_utils/generic_utils.ipynb
%run py_utils/sqlite_utils.ipynb

def get_albums(start_urls):
    album_dict = {}    
    a_key = 'a'
    div_key = 'div'
    artist_key = 'title'
    href_attr = {'class': 'listalbum-item'}
    album_attr = {'class': 'album'}
    tracks_table = {'id': 'listAlbum'}
    album_class = ['album']
    track_class = ['listalbum-item']
    root_url = "https://www.azlyrics.com"  
    
    for url in start_urls:
        print(">> Scraping {}".format(url))
        soup = get_request(url)        
        try:
            parse_table = bs4_scrape(soup, 'find', div_key, tracks_table)
            parse_list = bs4_scrape(parse_table, 'findAll', div_key)    

        except Exception as exc:
            print('! Exception: {}'.format(exc))
            single_list = bs4_scrape(soup, 'findAll', div_key, href_attr)
            parse_list = [item for item in single_list]
            
        artist_name = ' '.join(bs4_scrape(soup, 'find', artist_key).text.split(' ')[:-1])        
        album_name = []; album_year = []; track_name = []; track_url = []           
        get_album = ''; get_year = '' 
        
        for row in parse_list:  
            get_class = row.get('class') 
            
            if get_class == album_class: 
                try: 
                    get_album = ' '.join(row.text.split(' ')[:-1]).replace('"', '')  
                    get_year = int(''.join(row.text.split(' ')[-1]).replace('(','').replace(')',''))
                    
                except Exception as exc: 
                    print("! ValueError: {}".format(exc))
                    get_year = ''
                                
            elif get_class == track_class:
                track_name.append(row.text)
                track_url.append((lambda url: root_url+url if root_url not in url else url)
                    ((lambda url_tag: url_tag.get('href') if url_tag is not None else '')(bs4_scrape(row, 'find', a_key))))
                
                album_name.append(get_album if get_album != '' else 'other')
                album_year.append(get_year if get_year != '' else np.nan)
            
        
        album_dict[artist_name] = {'track_names': track_name, 'track_urls': track_url, 
                                   'album_names': album_name, 'album_years':album_year}
    return album_dict
